In [1]:
#Importamos los paquetes

In [2]:
import pandas as pd
import requests
import json
import numpy as np
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [3]:
#Obtenemos la tabla de Wikipedia directamente a un DataFrame

In [4]:
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

In [5]:
#Separamos los códigos de las direcciones y eliminamos los que no están asignados

In [6]:
postal_code = []
status = []
for row in data.iterrows():
    for i in row[1]:
        postal_code.append(i[:3])
        status.append(i[3:])

In [7]:
inte = pd.DataFrame({"Postal Code":postal_code,"Status":status})
inte = inte[inte.Status != "Not assigned"]

In [8]:
#Separamos de las direcciones el Borough del Neighborhood. Luego limpiamos el DataFrame y corregimos algunos typos
#SUPONGO QUE HAY QUE CORREGIRLOS DE LA MANERA EN QUE LO HAGO

In [9]:
inte["Borough"] = [i[0] for i in inte.Status.str.split("(")]
inte["Neighborhood"] = [",".join(i[1][:-1].split("/")) for i in inte.Status.str.split("(")]
inte.reset_index(inplace = True)
inte.drop(["index","Status"],axis = 1, inplace = True)

In [10]:
inte["Neighborhood"].replace(["Don Mills)Nort","Don Mills)Sout","Downsview)East","Downsview)Wes","Downsview)Centra","Willowdale)Sout","Downsview)Northwes","Willowdale)Wes"],
                             ["Don Mills North","Don Mills South","Downsview East","Downsview West","Downsview Central","Willowdale South","Downsview Northwest","Willowdale West"],
                             inplace = True)

In [11]:
inte["Borough"].replace(["East YorkEast Toronto","Mississauga, Canada Post Gateway Processing Centre","Downtown TorontoStn A PO Boxes25 The Esplanade","EtobicokeNorthwest","East TorontoBusiness reply mail Processing Centre969 Eastern"],
                        ["East York, East Toronto","Mississauga, Canada Post Gateway Processing Centre","Downtown Toronto, Stn A PO Boxes, 25 The Esplanade","Etobicoke Northwest","East Toronto, Business reply mail Processing Centre, 969 Eastern"],
                        inplace = True)

In [12]:
#RESULTADO

In [13]:
inte.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [14]:
inte.shape

(103, 3)

In [15]:
#Cargamos el fichero .CSV

In [16]:
geo_data = pd.read_csv("Geospatial_Coordinates.csv",header = 0)
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
#Juntamos los dos DataFrames usando como clave el código postal

In [18]:
df = inte.merge(geo_data, on = "Postal Code")
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [19]:
#RESULTADO

In [20]:
df.shape

(103, 5)

In [21]:
#AGRUPAMOS LOS VECINDARIOS

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name, lat, lng)
            
        # crear la URL de solicitud de API
        url = "https://api.foursquare.com/v3/places/search?ll={}%2C{}&radius={}&limit=50".format(lat, lng, radius)

        headers = {
            "Accept": "application/json",
            "Authorization": "fsq3HnELw/QsnnAagzVSpgUVF0Hq+X35SnL3wjbsWO8eC34="
                }

        response = requests.get(url, headers=headers)
        results = json.loads(response.text)["results"]
        # regresa solo información relevante de cada sitio cercano
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['geocodes']['main']['latitude'], 
            v['geocodes']['main']['longitude'],  
            v['categories'][0]['name']) for v in results if len(v['categories'])>0 and len(v['geocodes'])>0])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
toronto_venues = getNearbyVenues( df["Neighborhood"], df["Latitude"], df["Longitude"])

Parkwoods 43.7532586 -79.3296565
Victoria Village 43.7258823 -79.3155716
Regent Park , Harbourfront 43.6542599 -79.3606359
Lawrence Manor , Lawrence Heights 43.718518 -79.4647633
Ontario Provincial Government 43.6623015 -79.3894938
Islington Avenue 43.6678556 -79.5322424
Malvern , Rouge 43.8066863 -79.1943534
Don Mills North 43.7459058 -79.352188
Parkview Hill , Woodbine Gardens 43.7063972 -79.309937
Garden District, Ryerson 43.6571618 -79.3789371
Glencairn 43.709577 -79.4450726
West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale 43.6509432 -79.5547244
Rouge Hill , Port Union , Highland Creek 43.7845351 -79.1604971
Don Mills South 43.7258997 -79.340923
Woodbine Heights 43.6953439 -79.3183887
St. James Town 43.6514939 -79.3754179
Humewood-Cedarvale 43.6937813 -79.4281914
Eringate , Bloordale Gardens , Old Burnhamthorpe , Markland Wood 43.6435152 -79.5772008
Guildwood , Morningside , West Hill 43.7635726 -79.1887115
The Beaches 43.6763574 -79.2930312
Berczy Park 43

In [24]:
print(toronto_venues.shape)
toronto_venues.head()

(3873, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.754751,-79.328439,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751978,-79.333389,Food and Beverage Retail
2,Parkwoods,43.753259,-79.329656,Money in Motion,43.752947,-79.332418,Financial Service
3,Parkwoods,43.753259,-79.329656,Toronto Custom Lights,43.752947,-79.332418,Business and Professional Services
4,Parkwoods,43.753259,-79.329656,Pneutrans Systems Ltd,43.754856,-79.327753,Business and Professional Services


In [25]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighborhood,ATM,Accounting and Bookkeeping Service,Adult Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Amusement Park,...,Vintage and Thrift Store,Warehouse / Wholesale Store,Waste Management Service,Website Designer,Wholesaler,Wine Bar,Wine Store,Women's Store,Yoga Studio,Youth Organization
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,ATM,Accounting and Bookkeeping Service,Adult Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Amusement Park,...,Vintage and Thrift Store,Warehouse / Wholesale Store,Waste Management Service,Website Designer,Wholesaler,Wine Bar,Wine Store,Women's Store,Yoga Studio,Youth Organization
0,Agincourt,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
1,"Alderwood , Long Branch",0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
2,"Bathurst Manor , Wilson Heights , Downsview North",0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
3,Bayview Village,0.023256,0.093023,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.023256,0.0,0.000000,0.023256,0.0,0.0,0.0,0.0,0.000000,0.0
4,"Bedford Park , Lawrence Manor East",0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.021739,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,Willowdale West,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
98,Woburn,0.000000,0.052632,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.052632,0.0,0.0,0.0,0.0,0.000000,0.0
99,Woodbine Heights,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.021277,0.0
100,"York Mills , Silver Hills",0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0


In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

In [29]:
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
model = KMeans(n_clusters=5, random_state=148)
model.fit(toronto_grouped_clustering)

KMeans(n_clusters=5, random_state=148)

In [30]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', model.labels_)
toronto_merged = df
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [31]:
toronto_merged.drop(toronto_merged[toronto_merged["Cluster Labels"].isna() == True].index,inplace = True)

In [32]:
map_clusters = folium.Map(location=[43.651070, -79.347015], zoom_start=11)
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    cluster = int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters